In [41]:
#bibliotecas

import requests
import zipfile
import io
import pandas as pd
import numpy as np
from difflib import get_close_matches

In [43]:
# variaveis

start_y = 2011
final_y = 2022
codecvm = '019348'
#arquivo com data da publicação de relatórios
event_dates_file =  'c:\\Users\\thgcn\\OneDrive\\Academico\\Financial-Reports-Impact\\data\\itr_date_itau.csv'
#arquivo com preços históricos
itau_file = 'c:\\Users\\thgcn\\OneDrive\\Academico\\Financial-Reports-Impact\\data\\\historical_data\\ITUB4.SA.csv'


In [59]:
# funções 

# 1- captura da data de publicação

def busca_itr(ano):
    """
    Verifica se o diretório informado pelo usuário é válido.

    Returns:
        str: O caminho absoluto do diretório válido.

    Example:
        >>> verificar_diretorio()
        Digite o caminho do diretório: /caminho/do/diretorio
        Diretório raiz para armazenamento dos documentos: /caminho/do/diretorio
        '/caminho/do/diretorio'

    """
    url = 'https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/ITR/DADOS/itr_cia_aberta_%d.zip' % ano
    file = 'ipe_cia_aberta_%d.zip' % ano
    r = requests.get(url)
    zf = zipfile.ZipFile(io.BytesIO(r.content))
    file = zf.namelist()
    zf = zf.open(file[0])
    lines = zf.readlines()
    lines = [i.strip().decode('ISO-8859-1') for i in lines]
    lines = [i.split(';') for i in lines]
    qtl = len(lines)
    return lines

def save_cvm_dates_combined(start_y, final_y, codecvm, event_dates_file):
    combined_data = pd.DataFrame()  # DataFrame vazio para acumular os resultados
    
    # Loop para chamar a função para os anos de start_y a final_y
    for year in range(start_y, final_y + 1):
        # Suponha que 'busca_itr(year)' seja uma função que retorna os dados para o ano especificado
        data = busca_itr(year)
        
        # Criar um DataFrame a partir dos dados
        df = pd.DataFrame(data[1:], columns=data[0])
        
        # Filtrar os dados para o CD_CVM específico (codecvm)
        filtered_data = df.loc[df['CD_CVM'] == codecvm]
        
        # Selecionar apenas as colunas CD_CVM e DT_RECEB
        filtered_data = filtered_data[['CD_CVM', 'DT_RECEB']]
        
        # Adicionar os dados filtrados ao DataFrame acumulado
        combined_data = pd.concat([combined_data, filtered_data], ignore_index=True)
    
    # Salvar os dados acumulados como um único arquivo CSV
    combined_data.to_csv(event_dates_file, index=False)
    print(f'Dados combinados (CD_CVM e DT_RECEB) salvos em {event_dates_file}')    
    combined_data = pd.DataFrame()  # DataFrame vazio para acumular os resultados
    
def process_itau_data(itau_file, event_dates_file, codecvm):
    # Carregar dados do arquivo ITUB4.SA.csv
    itau_data = pd.read_csv(itau_file)
    itau_data['Date'] = pd.to_datetime(itau_data['Date'])  # Converter a coluna Date para o tipo datetime

    # Calcular o retorno diário em escala logarítmica
    itau_data['Return'] = np.log(itau_data['Close'] / itau_data['Close'].shift(1))

    # Carregar datas de eventos do arquivo itr_date_itau.csv
    event_dates = pd.read_csv(event_dates_file)
    event_dates['DT_RECEB'] = pd.to_datetime(event_dates['DT_RECEB'])  # Converter a coluna Date para o tipo datetime

    # Definir as datas de eventos como um conjunto para busca eficiente
    event_set = set(event_dates['DT_RECEB'])

    # Determinar se cada data é um evento (1) ou não (0)
    itau_data['event'] = itau_data['Date'].isin(event_set).astype(int)

    # Receber o codcvm via parâmetro de entrada da função
    itau_data['CD_CVM'] = codecvm

    # Selecionar as colunas relevantes para o DataFrame final
    final_data = itau_data[['CD_CVM', 'Date', 'Return', 'event']]
    
    return final_data 

def analyze_data_quality(final_data):
    # Identificar quantos "COD_CVM" distintos existem na base e exibir os valores
    unique_cod_cvm = final_data['CD_CVM'].unique()
    num_unique_cod_cvm = len(unique_cod_cvm)
    print(f"Existem {num_unique_cod_cvm} COD_CVM distintos na base:")
    print(unique_cod_cvm)
    
    # Identificar quantas datas iguais existem em "Date" e exibir os valores
    date_counts = final_data['Date'].value_counts()
    duplicate_dates = date_counts[date_counts > 1]
    print(f"\nExistem {len(duplicate_dates)} datas repetidas na base:")
    print(duplicate_dates)
    
    # Contar quantos eventos iguais a 0 e 1 existem na base
    event_0_count = (final_data['event'] == 0).sum()
    event_1_count = (final_data['event'] == 1).sum()
    print(f"\nContagem de eventos:")
    print(f"Eventos iguais a 0: {event_0_count}")
    print(f"Eventos iguais a 1: {event_1_count}")
    
    # Contar quantas vezes o evento = 1 em um intervalo de 1 ano e exibir as datas
    final_data['Year'] = final_data['Date'].dt.year
    events_by_year = final_data[final_data['event'] == 1].groupby('Year')['Date'].apply(list).reset_index()
    for _, row in events_by_year.iterrows():
        year = row['Year']
        event_dates = row['Date']
        print(f"\n{year}: {len(event_dates)} eventos")
        for date in event_dates:
            print(date.strftime('%Y-%m-%d'))
    
    # Identificar quais colunas contêm valores NaN e quantas vezes
    nan_counts = final_data.isna().sum()
    columns_with_nan = nan_counts[nan_counts > 0]
    print(f"\nColunas com valores NaN e a quantidade de NaNs:")
    print(columns_with_nan)

In [56]:
#chamada da função

save_cvm_dates_combined(start_y, final_y, codecvm, event_dates_file)
final_data = process_itau_data(itau_file, event_dates_file, codecvm)

Dados combinados (CD_CVM e DT_RECEB) salvos em c:\Users\thgcn\OneDrive\Academico\Financial-Reports-Impact\data\itr_date_itau.csv


In [61]:
# Chamada da função para analisar a qualidade dos dados
analyze_data_quality(final_data)

Existem 1 COD_CVM distintos na base:
['019348']

Existem 0 datas repetidas na base:
Series([], Name: Date, dtype: int64)

Contagem de eventos:
Eventos iguais a 0: 3186
Eventos iguais a 1: 39

2011: 4 eventos
2011-05-03
2011-08-02
2011-08-03
2011-11-01

2012: 4 eventos
2012-04-24
2012-07-24
2012-10-23
2012-10-30

2013: 3 eventos
2013-04-30
2013-07-30
2013-10-29

2014: 3 eventos
2014-04-29
2014-08-05
2014-11-04

2015: 3 eventos
2015-05-05
2015-08-04
2015-11-03

2016: 3 eventos
2016-05-03
2016-08-02
2016-10-31

2017: 4 eventos
2017-05-03
2017-07-31
2017-08-11
2017-10-30

2018: 2 eventos
2018-07-30
2018-10-29

2019: 4 eventos
2019-05-02
2019-07-29
2019-11-04
2019-11-25

2020: 3 eventos
2020-05-04
2020-08-03
2020-11-03

2021: 3 eventos
2021-05-03
2021-08-02
2021-11-03

2022: 3 eventos
2022-05-09
2022-08-08
2022-11-10

Colunas com valores NaN e a quantidade de NaNs:
Return    1
dtype: int64
